In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer 
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SpatialDropout1D , Dropout , Conv1D , MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping

In [ ]:
tweet = pd.read_csv('../input/twitter-airline-sentiment/Tweets.csv')
tweet.info()

In [ ]:
tweet = tweet[['airline_sentiment' , 'text']]

In [ ]:
tweet.info()

In [ ]:
sns.countplot(x = 'airline_sentiment' , data = tweet)
plt.show()

In [ ]:
tweet['text']

In [ ]:
sentence = [tweet['text'][2] , tweet['text'][14636]]
sentence

In [ ]:
appos = {
"aren't" : "are not",
"can't" : "cannot",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'd" : "I had",
"i'll" : "I will",
"i'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":" will",
"didn't": "did not"
}

In [ ]:
def cleanText(text):
    regx_clean = " ".join([re.sub(r'^@[a-zA-z]*' , "" , i) for i in text.split()])
    http_clean = " ".join([word for word in regx_clean.split() if 'http' not in word and word != 'RT'])
    tokens_ = word_tokenize(http_clean)
    words_ = [word for word in tokens_ if word.isalpha()]
    words_ = [w.lower() for w in words_]
#     stop_words = set(stopwords.words('english'))
#     remove_stop = [w for w in words_ if not w in stop_words]
    reformed = [appos[word] if word in appos else word for word in words_]
#     porter = PorterStemmer()
    porter = SnowballStemmer("english")
    stemmed = " ".join([porter.stem(word) for word in reformed])
    return stemmed
print(cleanText("@VirginAmerica What? @dhepburn! said i. 9124 , http.//hiSatyam"))

In [ ]:
text_data = tweet['text']
text_label = tweet['airline_sentiment']

In [ ]:
Lbl = LabelEncoder()
y = Lbl.fit_transform(text_label)

In [ ]:
y = to_categorical(y)

In [ ]:
X = []
for items in text_data:
    X.append(cleanText(items))

In [ ]:
X[:10]

In [ ]:
commonWord = ' '.join(X)


In [ ]:
from wordcloud import WordCloud,STOPWORDS
wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color='black',
                      width=3000,
                      height=2500
                     ).generate(commonWord)

In [ ]:
plt.figure(1,figsize=(12, 12))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
# y_train_n = to_categorical(y_train_n)
# token = Tokenizer(oov_token="<OOV>")
# token.fit_on_texts(x_train_n)
# word_index = token.word_index
# seq = token.texts_to_sequences(x_train_n)
# train_padding = pad_sequences(seq , maxlen = 50 , padding = 'pre')
# train_padding.shape

In [ ]:
token = Tokenizer(oov_token="<OOV>")
token.fit_on_texts(X)
word_index = token.word_index
seq = token.texts_to_sequences(X)
train_padding = pad_sequences(seq , maxlen = 50 , padding = 'pre')
train_padding.shape

In [ ]:
EMBEDDING_DIM = 200
model = Sequential()
model.add(Embedding(30000, EMBEDDING_DIM, input_length=train_padding.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(128, dropout=0.2,return_sequences=True))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(32))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:
epochs = 20
batch_size = 64
model.fit(train_padding, y, 
          epochs=epochs, 
          batch_size=batch_size,
          validation_split=0.2)

In [ ]:
# os.mkdir('./model')

In [ ]:
model.save('./model/twitterUSAirline.h5')

In [ ]:
new = ['it was a good experince']
    
seq = token.texts_to_sequences(new)
padded = sequence.pad_sequences(seq, maxlen=50)
pred = model.predict(padded)
classes  = np.argmax(pred , axis = -1)
dict_sentiment = {0 : 'Negative' , 1 : 'Positive' , 2 : 'Neutral'}
print(f'{new} : {dict_sentiment[int(classes)]}')